In [76]:
from nilearn import datasets
import pandas as pd
from nilearn import plotting
from nilearn.interfaces.fmriprep import load_confounds_strategy
from nilearn.maskers import NiftiMapsMasker,NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
import os
from pathlib import Path
import numpy as np
import abagen
from dipy.io.image import load_nifti
import nibabel as nib

In [77]:
freesurfer_path ="/workspace/home/datasets/mri/schizo_datasets/COBRE/COBRE_freesurfer/"

In [78]:
path =Path(freesurfer_path) / 'A00025969/mri/FS_label.nii.gz'

In [79]:
fs_prefix ='FS_label.nii.gz'

In [80]:
atlas = abagen.fetch_desikan_killiany()

In [81]:
atlas['image']

'/opt/miniconda-latest/lib/python3.8/site-packages/abagen/data/atlas-desikankilliany.nii.gz'

In [82]:
desikan_killiany_info =pd.read_csv(atlas['info'])

In [83]:
# Locate the data of the first subject
data_path = '/workspace/home/datasets/mri/schizo_datasets/COBRE/COBRE_fmriprep/'

In [84]:
#atlas = datasets.fetch_atlas_schaefer_2018()

In [85]:
def preproc_atlas(atlas_data):
    new_atlas_data =atlas_data.copy()
    for elem, new_elem in zip(np.unique(atlas_data),list(range(np.unique(atlas_data).shape[0]))):
        new_atlas_data[new_atlas_data==elem]=new_elem
    return new_atlas_data 

In [86]:
# Define plot function
def show_slices(image, axis1="x", axis2="y", axis3="z"):
    slice_0 = image[round(len(image[0])/2), :, :]
    slice_1 = image[:, round(len(image[1])/2), :]
    slice_2 = image[:, :, round(len(image[2])/2)]
    image = ([slice_0, slice_1, slice_2])
    fig, axes = plt.subplots(1, len(image), figsize=[15,15])
    for i, slice in enumerate(image):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")
        axes[0].set(xlabel=axis2, ylabel=axis3)
        axes[1].set(xlabel=axis1, ylabel=axis3)
        axes[2].set(xlabel=axis1, ylabel=axis2)
    plt.show()
    
def flatten_freesurfer_stats(path):
    """
        Parses FreeSurfer statistics then flattens included table of brain structure features into 1-row table as DataFrame.
    """

    from os.path import basename
    import pandas as pd
    from plumbum.colors import fatal

    try:
        # Get line with column names from file of FreeSurfer statistics.
        ColHeader = next(filter(lambda line: line.startswith('# ColHeaders'), open(path).readlines()))
        # Ignore first '#', 'ColHeaders'. Leave column names only.
        names = ColHeader.split()[2:] 
        data = pd.read_csv(path, names=names, comment='#', delim_whitespace=True, dtype=str)
#         columns = [col for col in filter(
#             lambda column: column not in ['Index', 'SegId', 'StructName'],
#             data.columns
#         )]
#         StructNames = data['StructName']
#         data = pd.DataFrame(data[columns].T.values.flatten()).T

#         # Get prefix (e.g. 'rh') from file name like ``rh.aparc.stats``
#         # as suffix for names of 1-row columns had been flattened from DataFrame.
#         filename = basename(path).split('.')
#         suffix = [filename[0]] if len(filename) > 2 else [] # There is no prefix from file name like ``aseg.stats``
#         data.columns = ['_'.join([prefix, infix] + suffix) for infix in columns for prefix in StructNames]
        return data[['SegId','StructName']]
    except StopIteration:
        print(fatal | "There is no header with column names starting with ColHeaders in the file", path)
    except FileNotFoundError as e:
        print(fatal | str(e))
    except KeyError as e:
        print(fatal | "There is no column", str(e))
    return pd.DataFrame()    

In [87]:
def get_dtk_labels(atlas_data):
    data =flatten_freesurfer_stats('column_names.txt')
    data['SegId'] =data['SegId'].astype(int)
    data=data.drop_duplicates(subset=['SegId'])
    labels =data.loc[data.SegId.isin(np.unique(atlas_data).tolist())]['StructName'].to_list()
    return labels

In [88]:
def extract_correlation_pipe(func,subj, atlas):
    print("extraction: ")
    # Loading atlas image stored in 'maps'
    atlas_filename = Path(freesurfer_path) / f'{subj.split("-")[-1]}/mri/aparc.DKTatlas+aseg.nii'
    # Loading atlas data stored in 'labels'
    
    atlas_data, affine, _ =load_nifti(atlas_filename, return_img=True)
    
    labels =get_dtk_labels(atlas_data)[1:]
    print(atlas_data.shape)
    print(labels)
    print("labels: ", len(labels))
    
    
    confounds_simple, mask =load_confounds_strategy(func)

    # masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True,
    #                      memory='nilearn_cache', verbose=5)
    # from nilearn.maskers import NiftiLabelsMasker
    masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True, resampling_target='labels',
                           memory='nilearn_cache', verbose=5)
    
    time_series = masker.fit_transform(func, confounds =confounds_simple)
    print(time_series.shape)
    
    correlation_measure = ConnectivityMeasure(kind='correlation')
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]
    np.fill_diagonal(correlation_matrix, 0)
    
    print(correlation_matrix.shape)
    data =pd.DataFrame(correlation_matrix, columns=labels, index =labels)
    corr_dict ={}
    for i, column in enumerate(data.columns):
        for j in data.columns[:i]:
            corr_value =data.loc[j,column]
            corr_dict[f'{j}-{column}'] =corr_value
    return  corr_dict,data, pd.DataFrame(time_series,columns=labels)

In [89]:
len(os.listdir(data_path))

333

In [ ]:
errrors =0
corr_array =[]
for subj in os.listdir(data_path):
            brain_path =Path(f'{data_path}/{subj}/func/{subj}_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')
            if brain_path.exists():
                try:
                    print(str(brain_path))
                    print("start")
                    corr_dict, data,time_series =extract_correlation_pipe(str(brain_path),subj, atlas)

                    time_series.to_csv(f"results/cobre_dk_big/{subj}_embed.csv")
                    data.to_csv(f"results/cobre_dk_big/{subj}.csv")
                    corr_dict["id"] =subj
                    corr_array.append(corr_dict)
                except Exception as e: 
                    print(e)

/workspace/home/datasets/mri/schizo_datasets/COBRE/COBRE_fmriprep/sub-A00000838/func/sub-A00000838_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
start
extraction: 
(256, 256, 256)
['Left-Cerebral-White-Matter', 'Left-Cerebral-Cortex', 'Left-Lateral-Ventricle', 'Left-Inf-Lat-Vent', 'Left-Cerebellum-White-Matter', 'Left-Cerebellum-Cortex', 'Left-Thalamus-Proper*', 'Left-Caudate', 'Left-Putamen', 'Left-Pallidum', '3rd-Ventricle', '4th-Ventricle', 'Brain-Stem', 'Left-Hippocampus', 'Left-Amygdala', 'CSF', 'Left-Accumbens-area', 'Left-VentralDC', 'Left-vessel', 'Left-choroid-plexus', 'Right-Cerebral-White-Matter', 'Right-Cerebral-Cortex', 'Right-Lateral-Ventricle', 'Right-Inf-Lat-Vent', 'Right-Cerebellum-White-Matter', 'Right-Cerebellum-Cortex', 'Right-Thalamus-Proper*', 'Right-Caudate', 'Right-Putamen', 'Right-Pallidum', 'Right-Hippocampus', 'Right-Amygdala', 'Right-Accumbens-area', 'Right-VentralDC', 'Right-vessel', 'Right-choroid-plexus', 'WM-hypointensities', 'Optic-Chiasm

In [ ]:
errrors

In [ ]:
dataset_corr =pd.DataFrame.from_dict(corr_array)

In [129]:
dataset_corr.to_csv("results/cobre_dk_func_connectivity.csv", index=False)

In [9]:
dataset_corr

,b'7Networks_LH_Vis_1'-b'7Networks_LH_Vis_2',b'7Networks_LH_Vis_1'-b'7Networks_LH_Vis_3',b'7Networks_LH_Vis_2'-b'7Networks_LH_Vis_3',b'7Networks_LH_Vis_1'-b'7Networks_LH_Vis_4',b'7Networks_LH_Vis_2'-b'7Networks_LH_Vis_4',b'7Networks_LH_Vis_3'-b'7Networks_LH_Vis_4',b'7Networks_LH_Vis_1'-b'7Networks_LH_Vis_5',b'7Networks_LH_Vis_2'-b'7Networks_LH_Vis_5',b'7Networks_LH_Vis_3'-b'7Networks_LH_Vis_5',b'7Networks_LH_Vis_4'-b'7Networks_LH_Vis_5',...,b'7Networks_RH_Default_PFCdPFCm_13'-b'7Networks_RH_Default_pCunPCC_9',b'7Networks_RH_Default_pCunPCC_1'-b'7Networks_RH_Default_pCunPCC_9',b'7Networks_RH_Default_pCunPCC_2'-b'7Networks_RH_Default_pCunPCC_9',b'7Networks_RH_Default_pCunPCC_3'-b'7Networks_RH_Default_pCunPCC_9',b'7Networks_RH_Default_pCunPCC_4'-b'7Networks_RH_Default_pCunPCC_9',b'7Networks_RH_Default_pCunPCC_5'-b'7Networks_RH_Default_pCunPCC_9',b'7Networks_RH_Default_pCunPCC_6'-b'7Networks_RH_Default_pCunPCC_9',b'7Networks_RH_Default_pCunPCC_7'-b'7Networks_RH_Default_pCunPCC_9',b'7Networks_RH_Default_pCunPCC_8'-b'7Networks_RH_Default_pCunPCC_9',id
0,0.469480,0.243579,-0.130661,0.297445,0.252037,0.233951,0.203479,0.028686,0.631804,0.234953,...,-0.183302,0.590225,0.310319,0.339426,0.484776,0.462425,0.585187,0.331663,0.432210,sub-A00000838
1,0.349527,0.166937,0.037780,0.518068,0.570200,-0.049268,0.423871,0.470621,0.279740,0.579940,...,0.202921,0.345601,0.262245,0.378454,0.136927,0.364337,0.379389,0.167012,0.243261,sub-A00036844
2,0.401065,0.482765,0.168628,0.387949,0.481772,0.220805,0.393426,0.423376,0.425554,0.660514,...,0.016336,0.457874,0.572041,0.523497,0.370256,0.563198,0.669595,0.481019,0.664976,sub-A00016197
3,0.310929,0.107777,0.046622,0.284062,0.422693,0.137627,0.198176,0.276744,0.316904,0.371812,...,0.226425,0.559681,0.465487,0.611576,0.522280,0.477719,0.679843,0.328467,0.632790,sub-A00006754
4,0.188564,0.492959,0.301123,0.502206,0.411595,0.700059,0.407204,0.272044,0.527731,0.699706,...,0.233161,0.478047,0.451229,0.492360,0.533554,0.612593,0.759704,0.582890,0.682155,sub-A00022509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,0.559692,0.295075,-0.029420,0.365114,0.167520,0.617846,0.218856,0.083400,0.713139,0.713665,...,0.355720,0.499800,0.285608,0.549731,0.437570,0.651632,0.662262,0.509567,0.634970,sub-A00036555
159,0.245285,0.306978,0.219735,0.324521,0.496356,0.313257,0.223951,0.156788,0.643565,0.323000,...,0.068176,0.551244,0.477161,0.596740,0.459070,0.609822,0.550925,0.372481,0.645473,sub-A00028408
160,0.451213,0.502780,0.265650,0.514819,0.470581,0.446471,0.439187,0.189752,0.717910,0.552771,...,0.138148,0.482856,0.070284,0.416024,0.144757,0.140928,0.061069,-0.058819,0.432118,sub-A00023848
161,-0.055583,0.066227,0.706332,0.303548,0.433949,0.620989,0.366222,0.205589,0.380377,0.542283,...,0.215773,0.062470,0.123570,0.384804,-0.009421,0.229665,0.215678,0.254070,0.271250,sub-A00024228
